This is my data visualization and exploration information
for data visualzation:
1. Group tweets by country if there is one
    - we can do this by using a list of constants that is every country in the world
    - Make sure to use states and abbreviations
2. Map hour of the day to retweet count
    - time zone isn't specified. Probaby U.T.C. which will be kind of wack
    - maybe this should go in the data exploration section
Correlation: Retweets
    - this is important because of poplaurity. Political campaign managers will want to know how to reach people
    1. Location/Country to retweet count
    2. hour to retweet count
    3. Keyword of top 100 words and retweet count - stop words - this is a big one
        - this is possible by getting means of words and how often they appear in a tweet vs retweet count
        - we can do this to a twitter uses description as well
        - we can also look at created at to see if accounts are established or not
            - we don't have follower numbers but we can assume older accounts probably have more followers
        - We can look at this difference between keywords and set differences between likes and retweets
            - retweets have more weight so we can see words that cause lots of likes but not a lot of retweets
                - This is calculated by percentages of likes vs retweets for anything that has over 10 likes/retweets

<h1> Data Understanding </h1>

In [40]:
import pandas as pd
import numpy as np

df = pd.read_csv('~/Downloads/australian-election-2019-tweets/auspol2019.csv')
print(df.describe())
df.head()

       retweet_count  favorite_count       user_id
count  183370.000000   183370.000000  1.833700e+05
mean        3.814310       11.159006  1.846078e+17
std        37.727466      118.324495  3.789751e+17
min         0.000000        0.000000  2.200000e+01
25%         0.000000        0.000000  1.002494e+08
50%         0.000000        1.000000  4.854974e+08
75%         1.000000        3.000000  2.981410e+09
max      6622.000000    15559.000000  1.130347e+18


,created_at,id,full_text,retweet_count,favorite_count,user_id,user_name,user_screen_name,user_description,user_location,user_created_at
0,2019-05-20 09:13:44,1130401208756187136,After the climate election: shellshocked green...,0.0,0.0,9.248486e+07,PIPELINEPETE,jocksjig,Retired Tradesman and Progressive Anti Conserv...,"Brisbane, Queensland",2009-11-25 09:19:45
1,2019-05-20 09:13:43,1130401205367140357,@narendramodi @smritiirani Coverage of indian ...,0.0,0.0,7.756474e+08,Narinder Parmar,nparmar1957,"Life coach & trainer, Motivational speaker, Ma...","Wollongong, NSW, AUSTRALIA",2012-08-23 10:20:40
2,2019-05-20 09:13:33,1130401162782371841,@workmanalice Do you know if Facebook is relea...,0.0,0.0,5.687300e+04,Peter Wells,peterwells,Writes for @theage and @smh on technology and ...,Melbourne,2006-12-11 07:38:06
3,2019-05-20 09:13:29,1130401143551434753,@vanbadham We all understand we have a compuls...,0.0,0.0,9.081660e+17,The Realist,therealist822,"Calls it as I see it. Anti PC, SJW and VS. If ...",NaN,2017-09-14 03:10:30
4,2019-05-20 09:13:23,1130401118666809345,"Shares were mixed in Asia, with India and Aust...",0.0,0.0,5.260074e+08,Inquirer Business,InquirerBiz,The official Twitter account of the Inquirer G...,Philippines,2012-03-16 03:51:59


In [41]:
print(df.dtypes)
print('===========')
print(df.info())

created_at           object
id                   object
full_text            object
retweet_count       float64
favorite_count      float64
user_id             float64
user_name            object
user_screen_name     object
user_description     object
user_location        object
user_created_at      object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183379 entries, 0 to 183378
Data columns (total 11 columns):
created_at          183379 non-null object
id                  183379 non-null object
full_text           183379 non-null object
retweet_count       183370 non-null float64
favorite_count      183370 non-null float64
user_id             183370 non-null float64
user_name           183358 non-null object
user_screen_name    183370 non-null object
user_description    167659 non-null object
user_location       146485 non-null object
user_created_at     183361 non-null object
dtypes: float64(3), object(8)
memory usage: 15.4+ MB
None


In [42]:
df.isna().sum(axis = 0, skipna = True)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140570 entries, 0 to 183378
Data columns (total 11 columns):
created_at          140570 non-null object
id                  140570 non-null object
full_text           140570 non-null object
retweet_count       140570 non-null float64
favorite_count      140570 non-null float64
user_id             140570 non-null float64
user_name           140570 non-null object
user_screen_name    140570 non-null object
user_description    140570 non-null object
user_location       140570 non-null object
user_created_at     140570 non-null object
dtypes: float64(3), object(8)
memory usage: 12.9+ MB


In [43]:

#this block is forcing all invalid datetimes and ids to be of the correct type
#before this i reccomend we show a matrix showing nil or invalid values
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
df["id"] = pd.to_numeric(df["id"], errors="coerce")
df["retweet_count"] = pd.to_numeric(df["retweet_count"], errors="coerce")
df["user_created_at"] = pd.to_datetime(df["user_created_at"], errors="coerce")
df["user_created_at"].min()

Timestamp('2006-03-21 21:04:12')

In [44]:
# describe each data type and its contents
desc = pd.DataFrame()
desc["Column"] = df.columns

desc["Description"] = [
    "Date on which the tweet was posted",
    "ID of the tweet",
    "Full tweet text",
    "Number of retweets",
    "Number of likes",
    "User ID of user who posted the tweet",
    "Username of user who posted the tweet",
    "Screen name of user who posted the tweet",
    "Free text description on profile of user who posted the tweet",
    "Free text location on profile of user who posted the tweet",
    "Date of create of account of user who posted the tweet"
]
#date is ordinal

desc["Scales"] = ['ordinal'] + ['nominal'] + ['ordinal'] + ['ratio'] * 2 + ['nominal'] *5 + ['ordinal'] 
#find out ranges
#there are some dates that aren't date ranges
#todo: Before coercing this values we should see everything that we have (or maybe after for seeing if it is a lot)

desc["Ranges"] = [
    "{} - {}".format(df['created_at'].min(), df['created_at'].max()),
    "{} - {}".format(df['id'].min(),df['id'].max()),
    "N/A",
    "0 - {}".format(df['retweet_count'].max()),
    "0 - {}".format(df['favorite_count'].max()),
    "N/A",
    "N/A",
    "N/A",
    "N/A",
    "N/A",
    "{} - {}".format(df["user_created_at"].min(), df["user_created_at"].max())
]
desc

,Column,Description,Scales,Ranges
0,created_at,Date on which the tweet was posted,ordinal,2019-05-10 16:58:15 - 2019-05-20 11:40:07
1,id,ID of the tweet,nominal,1126894229207171072 - 1130438045310672899
2,full_text,Full tweet text,ordinal,N/A
3,retweet_count,Number of retweets,ratio,0 - 6622.0
4,favorite_count,Number of likes,ratio,0 - 15559.0
5,user_id,User ID of user who posted the tweet,nominal,N/A
6,user_name,Username of user who posted the tweet,nominal,N/A
7,user_screen_name,Screen name of user who posted the tweet,nominal,N/A
8,user_description,Free text description on profile of user who p...,nominal,N/A
9,user_location,Free text location on profile of user who post...,nominal,N/A


In [45]:
# find the duplicate instances
print(len(df))
df.drop_duplicates(subset ="id", 
                     keep = False, inplace = True) 
print(len(df))

140570
140570


<h1>SECTION 3 ROUGH DRAFT </h1>
Data visualization
We're mainly going to be exploring retweets and likes 
We're going to assume utc on all times since datetime is not specified.
Check https://developer.twitter.com/en/docs/ads/general/guides/timezones.html for proof that twitter stores in utc
 were going to.
 1. gather data with created at tweet date and retweet count and like count
  - group by day for each
  - Take the day with the most retweets and do a group by hour on that time
 2. then we are going to look at account created at retweet count and like count
   - we can do a group by year here



In [46]:
#df.groupby(by=[df.created_at.month, df.created_at.year])
#turn these into graphs
import plotly.graph_objs as go
import plotly.express as px
tips = px.data.tips()
groups = df.set_index('created_at').groupby(pd.Grouper(freq='D')).mean()
strs = [pd.to_datetime(i).strftime("%B %d, %Y") for i in groups.index.values]
fig = go.Figure(data = [
    go.Bar(
        name='Retweets',
        x= strs, # assign x as the dataframe column 'x'
        y=groups['retweet_count']
    ),
    go.Bar(
        name='Likes',
        x=strs, # assign x as the dataframe column 'x'
        y=groups['favorite_count']
    )
])
fig.update_layout(barmode='stack')
fig.show()

In [47]:
groups = (
    df.query('created_at >= "May 11 00:00:00 2019" & created_at <= "May 11 23:59:59 2019"')
    .set_index('created_at')
    .groupby(pd.Grouper(freq='H'))
    .count()
)
#might want to do this for the 11th We dont have a full set for the 10th. SHOW THIS IN THE PREVIOUS SECTION


In [48]:
#i picked this day since it was the "FULL DAY" with  the highest retweet count
#this section shows which hours UTC have the highest mean retweet count
#we can see that it is at 1 AM UTC which is between  9 AM and 11 AM in Australia depending on Time Zone. (NO DST)
#11AM based on the time in Sydney
#NOTE WE MIGHT WANT TO NORMALIZE THIS
(
    df.query('created_at >= "May 11 00:00:00 2019" & created_at <= "May 11 23:59:59 2019"')
    .set_index('created_at')
    .groupby(pd.Grouper(freq='H'))
    .mean()
)
strs = [pd.to_datetime(i).strftime("%H") for i in groups.index.values]
fig = go.Figure(data = [
    go.Bar(
        name='Retweets',
        x= strs, # assign x as the dataframe column 'x'
        y=groups['retweet_count']
    ),
    go.Bar(
        name='Likes',
        x=strs, # assign x as the dataframe column 'x'
        y=groups['favorite_count']
    )
])
fig.update_layout(barmode='stack', title="Likes/Retweets per hour on May 11th")
fig.show()

In [49]:
#now group by year for created at
#NOTE: WE MIGHT WANT TO NORMALIZE THIS
groups = df.set_index('user_created_at').groupby(pd.Grouper(freq='Y')).count()
strs = [pd.to_datetime(i).strftime("%Y") for i in groups.index.values]

fig = go.Figure(data = [
    go.Bar(
        name='Retweets',
        x= strs, # assign x as the dataframe column 'x'
        y=groups['retweet_count']
    ),
    go.Bar(
        name='Likes',
        x=strs, # assign x as the dataframe column 'x'
        y=groups['favorite_count']
    )
])
fig.update_layout(barmode='stack', title="Total Likes/Retweets Based on user created at")
fig.show()

In [50]:
groups = df.set_index('user_created_at').groupby(pd.Grouper(freq='Y')).mean()
strs = [pd.to_datetime(i).strftime("%Y") for i in groups.index.values]

fig = go.Figure(data = [
    go.Bar(
        name='Retweets',
        x= strs, # assign x as the dataframe column 'x'
        y=groups['retweet_count']
    ),
    go.Bar(
        name='Likes',
        x=strs, # assign x as the dataframe column 'x'
        y=groups['favorite_count']
    )
])
fig.update_layout(barmode='stack', title="Average per year Likes/Retweets Based on user created at")
fig.show()

In [51]:
df['full_text'][0]

'After the climate election: shellshocked green groups remain resolute https://t.co/wyJzmAcyiD'

In [52]:
#now we are going to do text data on the tweet regarding the words and which words seem to get the most retweets 
#we will need to stem every word in every tweet and remove special characters 
#we will also remove stop words as well
import nltk
nltk.download('stopwords')
stop = nltk.corpus.stopwords.words('english')
#applying stop words... this may take a while
df['full_text'] = df['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#above line takes too fucking long! just remove the stop words from the matrix after we have our bag of words

[nltk_data] Downloading package stopwords to /Users/munch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
#stemming words... this is going to take a while
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()

#This code was taken from this very smart stackoverflow answer 
#https://stackoverflow.com/questions/43795310/apply-porters-stemmer-to-a-pandas-column-for-each-word
def stem_shorten(sentence):
    words = sentence.split()
    stemmed_tokens = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_tokens)

df['full_text'] = df['full_text'].apply(stem_shorten)
(df['full_text'][0])

'after climat election: shellshock green group remain resolut https://t.co/wyjzmacyid'

In [54]:
print(df['full_text'][11877])

australia decides: prime minist @scottmorrisonmp declar victori 2019 feder election. https://t.co/dsgfbimyw #ausvot #7new https://t.co/2ovamroiyu


In [55]:
import re
def regex(sentence):
    return re.sub(r'[^a-zA-Z#]', ' ', sentence)
#df['team'] =  df['team'].apply(lambda x: re.sub(r'[\n\r]*','', str(x)))
#this removes all special characters from every tweet
df['full_text'] = df['full_text'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', str(x)))
df['full_text'][0]

'after climat election  shellshock green group remain resolut https   t co wyjzmacyid'

In [56]:
#now that we have all of our stemming and regex applied, we can get our bag of words and our tf idf for retweets
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
#we should show this normalized and not normalized
#not sure how to do this correlation between text data and table data in one line
#so heres a hash that gets a word and adds its score from the retweet count
word_scores = {}
index = 0
for col in df['full_text']:
    for word in col.split():
        if word in word_scores:
            try:
                score = df['retweet_count'][index]
                word_scores[word] += score
            except:
                break
        else:
            score =  df['retweet_count'][index]
            word_scores[word] = score
    index += 1
    #now divide by the number of rows
word_scores.update((x, y/len(df.index)) for x, y in word_scores.items())
#get the highest words
    

KeyError: 3

In [ ]:
import operator
#MAYBE WE CAN USE A SWARM PLOT HERE?
sorted_d = sorted(word_scores.items(), key=operator.itemgetter(1), reverse=True)
#now we are displaying as bar graph
#we might want to use subplots for each 4 plots
import plotly.express as px
print(px.data.gapminder().query("country == 'Canada'").__class__.__name__)
#data_canada = px.data.gapminder().query("country == 'Canada'")
dfObj = pd.DataFrame(sorted_d[:100], columns = ['word' , 'score']) 
fig = px.bar(dfObj, x='word', y='score')
fig.show()
#rerun for stop words

In [ ]:
#now normalize the data by the amount of words in each tweet
word_scores = {}
index = 0
for col in df['full_text']:
    num_words = float(len(col.split()))
    for word in col.split():
        if word in word_scores:
            try:
                score = df['retweet_count'][index]
                word_scores[word] += (score / num_words)
            except:
                break
        else:
            score =  df['retweet_count'][index]
            word_scores[word] = (score / num_words)
    index += 1
    #now divide by the number of rows
word_scores.update((x, y/len(df.index)) for x, y in word_scores.items())

In [ ]:
sorted_d = sorted(word_scores.items(), key=operator.itemgetter(1), reverse=True)
dfObj = pd.DataFrame(sorted_d[:100], columns = ['word' , 'score']) 
fig = px.bar(dfObj, x='word', y='score')
fig.show()

In [ ]:
# this will stem and shorten every word in user description
nltk.download('stopwords')
stop = nltk.corpus.stopwords.words('english')
#applying stop words... this may take a while
df['user_description'] = df['user_description'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))

In [ ]:

df['user_description'] = df['user_description'].apply(stem_shorten)
(df['user_description'][0])

In [ ]:
df['user_description'] = df['user_description'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', str(x)))
df['user_description'][0]

In [ ]:
#now this section if for getting tfidf scores 
#WRITING THIS NOW SHOULD WE INCLUDE HASHTAGS?
#SCREW THE TFIDF This is about descriptions because tfidf does not seem super helpful compared to the last plot
word_scores = {}
index = 0
for col in df['user_description']:
    for word in str(col).split():
        if word in word_scores:
            try:
                score = df['retweet_count'][index]
                word_scores[word] += score
            except:
                break
        else:
            score =  df['retweet_count'][index]
            word_scores[word] = score
    index += 1
    #now divide by the number of rows
word_scores.update((x, y/len(df.index)) for x, y in word_scores.items())

In [ ]:
sorted_d = sorted(word_scores.items(), key=operator.itemgetter(1), reverse=True)
dfObj = pd.DataFrame(sorted_d[:100], columns = ['word' , 'score']) 
fig = px.bar(dfObj, x='word', y='score')
fig.show()

In [ ]:
# now normalized
word_scores = {}
index = 0
for col in df['user_description']:
    num_words = float(len(col.split()))
    for word in str(col).split():
        if word in word_scores:
            try:
                score = df['retweet_count'][index]
                word_scores[word] += (score / num_words)
            except:
                break
        else:
            score =  df['retweet_count'][index]
            word_scores[word] = (score / num_words)
    index += 1
    #now divide by the number of rows
word_scores.update((x, y/len(df.index)) for x, y in word_scores.items())

In [ ]:
sorted_d = sorted(word_scores.items(), key=operator.itemgetter(1), reverse=True)
dfObj = pd.DataFrame(sorted_d[:100], columns = ['word' , 'score']) 
fig = px.bar(dfObj, x='word', y='score')
fig.show()

<h1>DATA RELATIONSHIP</h1>
#now in this section we will explore correlations between location
This section will explore a correlation matrix and discuss IN WORDS their correlation
This section should not be tricky. It should be very similar to the graded example

In [ ]:
import seaborn as sea
import matplotlib.pyplot as p

cmap = sea.diverging_palette(240, 10, n=7, as_cmap=True)

f, ax = p.subplots(figsize=(9,9))
sea.heatmap(df.corr(), cmap=cmap, annot=True)

<h1>ADDITIONAL ANALYSIS</h1>
There is a very interesting section of this data which is location. However, location is never very clear because many users may abbreviate their location or not give a complete location    ex: U.S.A which is very broad location. I will explore how we will handle such cases in the future.   
Thankfully, Twitter provides a latitude and longitude for every location that is mentioned in these tweets.


In [ ]:
locs = pd.read_csv('~/Downloads/australian-election-2019-tweets/location_geocode.csv')
#lets combine loc with df to allow for quick indexing without cross-refrencing
#this may take a while
dframe = df.copy()
dframe["longitude"] = np.nan
dframe["latitude"] = np.nan
for index, row in locs.iterrows():
    dframe.loc[dframe.user_location == row["name"], ['latitude', 'longitude']] = [row['lat'], row['long']]
        #dframe.loc[dframe.user_location == row["name"], 'n'] = df2[df2.filename == 'test2.dat'].loc[0]['n']
        #dframe.loc[i, ['latitude', 'longitude']] = row[['lat', 'long']]
    #dframe.loc[["user_location"] == row["name"]
dframe

now that we have our imputed columns we can begin to look at correlations between location and retweet count
this first chart will be a basic total tweets per location
The second chart will be normalized retweets per location
I will be using the google maps api *gmplot* 
If you wish to render it please install the gmplot package 

In [ ]:
dframe.retweet_count = df.retweet_count.astype(int)
dframe

In [ ]:
import plotly.express as px
#this is really big so maybe we should only do the top 100
import plotly.graph_objects as go
fig = go.Figure(data=go.Scattergeo(
        locationmode = 'ISO-3',
        lon = dframe['longitude'],
        lat = dframe['latitude'],
        text = dframe['user_location'],
        mode = 'markers',
        marker = dict(
            size = 1,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'hot',
            cmin = 0,
            color = dframe['retweet_count'],
            cmax = df['retweet_count'].max(),
            colorbar_title="Incoming flights<br>February 2011"
        )))
fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo = dict(
            scope='world',
            projection_type='robinson',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 5,
            subunitwidth = 5
        ),
    )
fig.show()

This looks like a mess since there are hundreds of different locations. Let's just do the top 100 locations with the most cumulative tweets.

First. As a bar graph


In [ ]:
#the first thing we do is group by latitude and longitude for locations with the same latitude and longitude
#uncomment the below line when you need to rerun. It keeps killing the kernel
#top = dframe.groupby(['latitude', 'longitude']).count().sort_values(ascending=False, by='id').head(100)
#now we will grab these locations in our location dataframe
import plotly.express as px
loc_counts = []
for index, row in top.iterrows():
    #dframe.loc[dframe.user_location == row["name"], ['latitude', 'longitude']] = [row['lat'], row['long']]
    #df[(df[Gender]=='Male') & (df[Year]==2014)]
    location = (locs[(locs['lat'] == index[0]) & (locs['long'] == index[1])]).head(1).name
    s = (''.join([i for i in location if not i.isdigit()]))
    loc_counts.append([s, row[0]])
dfObj = pd.DataFrame(loc_counts, columns = ['location' , '# of tweets']) 
fig = px.bar(dfObj, x='location', y='# of tweets')
fig.show()

Now... As a map

In [ ]:
q = np.array(top.index.values, dtype=object)
#print(top.index.values.shape)
np.array(loc_counts)[:,0]

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scattergeo(
        locationmode = 'ISO-3',
        lon = np.array(top.axes[0].get_level_values(1)),
        lat = np.array(top.axes[0].get_level_values(0)),
        text = np.array(loc_counts)[:,0],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'solar',
            cmin = 0,
            color = top.retweet_count,
            cmax = top.retweet_count.max(),
            colorbar_title="Incoming flights<br>February 2011"
        )))
fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo = dict(
            scope='world',
            projection_type='robinson',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 5,
            subunitwidth = 5
        ),
    )
fig.show()

We can see here that most of our tweets, came from Australia. No surpirse there.   WE can also see that lots of tweets came from the U.S which is not surprising, as expected, most people reporting on the Australia election are either from Australia or from the larger cities in the U.S.    Probably because lots of major news centers are in the U.S. which find the Australian election important.
   if you render this notebook you can zoom in on certain locations and look at the heat maps for certain coordinates across the world.

This chart looks for matching longitude and latitudes from each grouping and dislays the total count of all locations with the same latitude and longitude. I grabbed the first name that appeared in respect to that latitude and longitude so that is why 'Central Victoria AUSTRALIA' is so high even with the strange casing. it is the same as the location 'Central Victoria'   
We can see that a lot of users simply put their locaton as 'Australia' or 'United States'. For the purpose of looking at raw counts I did not normalize this data set. In the next chart

Now for the interesting part. We are going to look at standardized retweet numbers across the globe. Which locations are most likely to recieve the highest amount of retweets out of the top 100 most tweeted from locations? This will be very important to news reporters who will want to conduct interviews in certain locations.

In [ ]:
top_locs = top.copy()
ind = dframe.latitude.isin(top_locs.axes[0].get_level_values(0)) & dframe.longitude.isin(top_locs.axes[0].get_level_values(1))
top_locs = dframe[ind]

In [ ]:
import operator
top_norm = top_locs.groupby(['latitude', 'longitude'])
top_norm = top_norm.first()[['retweet_count', 'favorite_count']].transform(lambda x: (x - x.mean()) / x.std())
loc_ret = []
for index, row in top_norm.head(100).iterrows():
    location = (locs[(locs['lat'] == index[0]) & (locs['long'] == index[1])]).head(1).name
    s = (''.join([i for i in location if not i.isdigit()]))
    loc_ret.append([s, row[0]])
sorted_d = sorted(loc_ret, key=operator.itemgetter(1), reverse=True)
dfObj = pd.DataFrame(sorted_d, columns = ['location' , '# of retweets']) 
fig = px.bar(dfObj, x='location', y='# of retweets')
fig.show()

After looking at the standardized information in a bar graph we can see something truly amazing. Any tweet with the same latitiude, longitude as capital hill, Canberra or the Parliament House has an EXTREMELY high chance of receiving a ton of retweets. This is not faulty because we see earlier that Capital Hill, Canberra ranks in the top 100 in regards to locations with the most amount of tweets. They have the same latitude, longitude in my csv file.   
This means that anyone who tweets with news directly from Capital Hill in Australia is extremely likely to recieve serious attention. 

Now lets look at this same correlation on an interactive map for a more pleasurable viewing experience.

In [ ]:
name = (top_locs.groupby(['latitude', 'longitude']).first()[['user_location']])
fig = go.Figure(data=go.Scattergeo(
        locationmode = 'ISO-3',
        lon = np.array(top_norm.axes[0].get_level_values(1)),
        lat = np.array(top_norm.axes[0].get_level_values(0)),
        text = np.array(name.values[:,0]),
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'solar',
            cmin = 0,
            color = top_norm.retweet_count,
            cmax = top_norm.retweet_count.max(),
            colorbar_title="Incoming flights<br>February 2011"
        )))
fig.update_layout(
        title = 'Standardized retweet counts per location',
        geo = dict(
            scope='world',
            projection_type='robinson',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 5,
            subunitwidth = 5
        ),
    )
fig.show()